# Calibration to market prices using ```pyop3```

Author: Caden Lee (caden.finsinyur@gmail.com)

In [1]:
import pyop3
from scipy.optimize import brentq
import scipy
from scipy import stats
import numpy as np

## Overview

Welcome to Tutorial 4 of the PyOptionTree tutorial series! 

We have come a long way since the first tutorial, when we started learning the basics of the ```pyop3``` functionalities, and understood the strengths and weaknesses of pricing European options using the tree methods. In the last tutorial, we took the step forward to work American option pricing using the ```pyop3```. This is a key value proposition of the Binomial Tree option pricing model - that it serves as a handy tool to price non-vanilla options which require explicitly solving each node along the path of the price evolution. Such options do not have a satisfactory closed-form solution; there are attempts to derive semi-analytical solutions that still require numerical approximation. There are some successes to derive a closed form solution for perpetual American put options, but having expiry at time infinite is no where close to realism. Hence, the binomial tree option pricing model still serves as a viable and simple approach to pricing American contracts.

In this tutorial, we will be delving into some possible practical use-cases of ```pyop3```, making use of functionalities we have introduced in earlier tutorials. As the title of the tutorial suggests, we will be focusing on:
- Calibration to market prices
- DeAmericanization

## Section 1: Calibration

In some situation, one may wish to price a complex options. For simpler options that are listed and traded on exchanges, prices are relatively transparent as there are market makers putting bid and offer quotes on screen. However, for complex options, it is rather difficult to arrive at the fair value of the payoff.

One of a few things option writers would do is to derive the implied volatility from the market prices of a listed option on a related underlying asset - this could be of the same underlying, or one that exhibitis high correlation with the target underlying asset. With the implied volatility, one variable is obtained and it allows option writer to then value the payoff based on it. This process is known as calibration.

The calibration process is fairly straightforward; when using a Black-Scholes model, one simply needs to find an implied volatility that, when applied with the Black-Scholes option pricing formula, gives a theoretical price that matches the observed market price. As the market usually comes with a bid price and an ask price, we usually attempt to match the mid-price of the bid-ask. The optimal implied volatility can be easily found using root-finding algorithms, such as the Brent's method or the Newton's method.

As the binomial tree option pricing model (specifically the CRR and RB trees) is a discrete version of the Black-Scholes model, the same calibration process could be applied. Recall from Tutorial 2 that the upward movement $u$ is related to the implied volatility of the option. In ```pyop3```, we attempt to fit the market price to the binomial tree option price, to arrive at the optimal $u$. As of ```pyop3``` v0.1.0, Brent's method is employed. 



### Calibrating European Options

Note: For the following calibration, we will be using Rendleman-Bartter Tree as opposed to Cox-Ross-Rubinstein Tree. Due to RB Tree's $u$ and $d$ parameters are more complex, we need to restrict the calibration range to between 1.0001 and 1.05. For CRR Tree, we could use the default range between 1.0001 to 10.0.

In [2]:
# Some market date - dated 1/12/2020

SPX_index = 3662.45 # spot price
zero_rate = 0.114128/100 # for simplicity, we used the 13-day zero rate instead of applying the interpolation
spot_date = '01/12/2020'
T = '18/12/2020'

SPX_put = 0.75
SPX_put_strike = 2675 # we chose an OTM put

SPX_call = 52.65
SPX_call_strike = 3665 # we chose an OTM call

In [3]:
calibrated_put_tree = pyop3.calibrate_european(SPX_put, SPX_index, SPX_put_strike, zero_rate, T,\
                                          spot_date = spot_date, N = 100, call = False, tree_type = "RB",\
                                              calibrate_range = (1.0001,1.05))

In [4]:
calibrated_put_tree.underlying_asset_summary()
print("Optimal u: ",calibrated_put_tree.u)

UNDERLYING ASSET SUMMARY
        +--------------------------------+
              Spot price: 	 $3662.45
              Time to expiry: 	 0.0516 years
              interest rate: 	 0.11%
              implied vol: 	 55.29%
              
No dividend payment expected during the course of the contract.
Optimal u:  1.0125576228549018


In [5]:
calibrated_call_tree = pyop3.calibrate_european(SPX_call, SPX_index, SPX_call_strike, zero_rate, T,\
                                          spot_date = spot_date, tree_type = "RB", N = 100,\
                                               calibrate_range = [1.0001, 1.05])

In [6]:
calibrated_call_tree.underlying_asset_summary()
print("Optimal u: ",calibrated_call_tree.u)

UNDERLYING ASSET SUMMARY
        +--------------------------------+
              Spot price: 	 $3662.45
              Time to expiry: 	 0.0516 years
              interest rate: 	 0.11%
              implied vol: 	 16.20%
              
No dividend payment expected during the course of the contract.
Optimal u:  1.0036807946484954


From the above calibration, we managed to derive the implied volatilities of the OTM put and call options respectively. As expected, the implied volatility of the OTM put is much higher than that of the OTM call, illustrating two points of a volatility skew.

As we learnt, the binomial tree option pricing model is a discrete representation of the Black-Scholes model. We should therefore get similar implied volatilities when attempting to calibrate using the analytical formula. Bearing in mind, from Tutorial 2 we concluded that there will be some pricing differences.

We shall attempt that and compare the implied volatilities.

In [7]:
# Create function to price vanilla options analytically using Black-Scholes Model
def black_scholes_option_pricer(S, K, r, T, sigma, call = True):
    '''
    Function calculates option prices based on Black-scholes model.
    Three modes available: Vanilla (default), Cash-or-Nothing (CON), Asset-or-Nothing (AON)
    
    Inputs:
    S: underlying asset price at t; can be an array of prices
    K: strike price; can be an array of prices
    r: interest rate, annualized
    T: time to expiration (also the T-t in our equations), in number of years
    sigma: implied volatility of the option
    call: default True. True if pricing call options; otherwise False

    Outputs:
    Option Prices.
    '''

    d2 = (np.log(S/K) +(r - 0.5*np.square(sigma))*(T))/(sigma*np.sqrt(T))
    d1 = d2 + sigma*np.sqrt(T)
    
    d2 = d2 if call == True else -d2
    d1 = d1 if call == True else -d1
    
    option_values = S*scipy.stats.norm.cdf(d1) - K*np.exp(-r*T)*scipy.stats.norm.cdf(d2)
    option_values = option_values if call == True else -option_values

    return option_values

In [8]:
sigma_put = brentq(lambda x:SPX_put- black_scholes_option_pricer(SPX_index,
                                                     SPX_put_strike, 
                                                     zero_rate, 
                                                     0.0516, 
                                                     x, 
                                                     call = False), 1e-12, 10.0)

print("Implied volatility of OTM put option obtained from BS model: {:.2f}%".format(sigma_put*100))

sigma_call = brentq(lambda x:SPX_call- black_scholes_option_pricer(SPX_index,
                                                     SPX_call_strike, 
                                                     zero_rate, 
                                                     0.0516, 
                                                     x, 
                                                     call = True), 1e-12, 10.0)

print("Implied volatility of OTM call option obtained from BS model: {:.2f}%".format(sigma_call*100))

Implied volatility of OTM put option obtained from BS model: 55.03%
Implied volatility of OTM call option obtained from BS model: 16.21%


As expected, the implied volatilties obtained from the Black-Scholes model is quite close to our calibrated values using the binomial tree model. From Tutorial 2, we also found out that the more out-of-the-money the option is, the higher the pricing error. This explains why the difference in the derived OTM put implied vols is higher than that of the OTM call, since the put strike is further from the spot that the call strike.

## Section 2: DeAmericanization

The above calibration also works for American options, by using the ```pyop3.calibrate_american()``` function, which takes in the same parameters as ```pyop3.calibrate_european()```. To avoid repetition, we will not demonstrate the calibration, and will leave to the learners to test out the calibration.

Now that we have introduced calibration, this leads us to a practical application known as "deAmericanization". According to Burkovaska et al, deAmericanization is the process of converting available American option data into pseudo-European option prices for further calibration. DeAmericanization offers advantage of simpler and fast calibration of American options and is a market standard.

DeAmericanization follows a 3-step process:
1) retrieving market prices of American options
2) simplification of American options into de-Americanized pseudo-European option prices via Binomial tree model, and 
3) calibrate Pseudo-European prices for implied volatility

In essence, the deAmericanization methodology is made up of the American option calibration process; firstly calibrate American options to get the implied $u$, and use the $u$ to derive the pseudo-European price.

In [9]:
# Some market date - dated 1/12/2020

SPY_index = 366.02 # spot price
zero_rate = 0.114128/100 # for simplicity, we used the 13-day zero rate instead of applying the interpolation
spot_date = '01/12/2020'
T = '18/12/2020'
div  = 1.58
ex_div_date = '18/12/2020'

SPY_put = 0.115
SPY_put_strike = 280 # we chose an OTM put

SPY_call = 3.445
SPY_call_strike = 370 # we chose an OTM call

In [21]:
pyop3.deamericanization(SPY_call, SPY_index, SPY_call_strike, zero_rate, T,\
                        spot_date = spot_date, N = 100, freq_by = "N", \
                        calibrate_range = (1.0001, 1.05), tree_type = "RB",\
                        div = div, ex_div_date = ex_div_date)

{'underlying_asset': <pyop3.binomial_tree.binomial_tree at 0x19741efbbc8>,
 'calibrated u': 1.0035536335229571,
 'calibrated impl. vol': 0.15643389082227477,
 'Equivalent European Option value': 2.92049889052248,
 'Early Exercise Premium': 0.5245011094775198}

In [31]:
pyop3.deamericanization(SPY_put, SPY_index, SPY_put_strike, zero_rate, T,\
                    spot_date = spot_date, N = 100, freq_by = "N", call = False, \
                    calibrate_range = (1.0001, 1.05), tree_type = "RB",\
                    div = div, ex_div_date = ex_div_date)

C:\Users\leeca\anaconda3\envs\finsinyur\lib\site-packages\pyop3\base_conditions.py:52: UserWarning: This message is a reminder to make sure that, since you defined ex_div_date,
                          the freq_by parameter is set to "days" instead of "N".
  the freq_by parameter is set to "days" instead of "N".''')


{'underlying_asset': <pyop3.binomial_tree.binomial_tree at 0x19742fe98c8>,
 'calibrated u': 1.0112084558711574,
 'calibrated impl. vol': 0.49347999200044484,
 'Equivalent European Option value': 0.11504488432638027,
 'Early Exercise Premium': -4.488432638026507e-05}

Voila! We have successfully executed our deAmericanization process to obtain a pseudo-European option price and the calibrated $u$ and implied vol parameters. 

It is important to note that under usual circumstance, an American call option would not have an early exercise premium as it is always optimal to exercise at expiry since one can earn the interest rate while holding the cash. Conversely, there will be an early exercise premium on American puts for the same reason.

In our example above, the reverse occurs. This is due to the presence of dividends. In order to gain the dividends, one would wish to exercise early to be entitled the dividend dollar. Conversely, dividends will bring down the underlying asset price post ex-dividend date, therefore it is optimal to wait till then before exercising the American put.

We are done with Tutorial 4! Do provide feedback on improvements or do reach out with questions. Thank you!

© Copyright 2023, Team PyOptionTree.